In [1]:
# project: p6
# submitter: jmaloney3
# partner: ccmaloney
# hours: 5

In [2]:
import csv

def process_csv(filename):
    example_file = open(filename, encoding="utf-8")
    example_reader = csv.reader(example_file)
    example_data = list(example_reader)
    example_file.close()
    return example_data

# Use process_csv to pull out the header and data rows
csv_rows = process_csv("airbnb.csv")
csv_header = csv_rows[0] # A list of the column headers
csv_data = csv_rows[1:] # The entire CSV data set besides the headers

def cell(row_idx, col_name):
    col_idx = csv_header.index(col_name)
    val = csv_data[row_idx][col_idx]
    if val == "":
        return None
    
    try: return int(val)
    except ValueError: pass
    
    try: return float(val)
    except ValueError: pass

    return val

def unique(l):
    return list(set(l))

def remove_none(l):
    return filter(lambda x: not x is None, l)

def coerce_str(l):
    return list(map(lambda x: str(x), l))

In [3]:
csv_header

['room_id',
 'name',
 'host_id',
 'host_name',
 'neighborhood_group',
 'neighborhood',
 'latitude',
 'longitude',
 'room_type',
 'price',
 'minimum_nights',
 'number_of_reviews',
 'last_review',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

In [4]:
#Q1: What neighborhood groups are present in the airbnb dataset?
#list(set(map(lambda row: cell(row[0], 'neighborhood_group'), enumerate(csv_data))))

def extract_column(name):
    l = []
    for row in enumerate(csv_data):
         l.append(cell(row[0], name))
    return l

unique(extract_column('neighborhood_group'))

['Manhattan', 'Brooklyn', 'Bronx', 'Staten Island', 'Queens']

In [5]:
#Q2: What is the average number of reviews among all the reviews?
from statistics import mean
round(mean(extract_column('number_of_reviews')))

23

In [6]:
#Q3: Count the number of rooms located in neighborhood "SoHo"?

len(
    list(
        filter(
            lambda nb: nb == "SoHo", 
            extract_column('neighborhood')
        )
    )
)


358

In [7]:
import re

def find_room_names(contained):
    return list(filter(lambda name: re.search(contained, str(name), re.IGNORECASE), 
           extract_column('name')))

def secondary_word_in_found_rooms(find_room_word, secondary_word):
    return list(
        filter(
            lambda name: re.search(find_room_word, str(name), re.IGNORECASE) and re.search(secondary_word, str(name), re.IGNORECASE), 
           extract_column('name')))

In [8]:
#Q4: Find the room names containing "MSG"
find_room_names("MSG")

['The Epic: Right by MSG',
 'Lavish 3 bedroom near Penn Station / MSG',
 'Stately Midtown Studio w/ Doorman, Gym, near MSG by Blueground',
 'Room Near Times Square/Javits/Hells Kitchen/MSG',
 'Cheery Midtown 1BR w/ Doorman + Gym, near MSG by Blueground',
 'Homey Hudson Yards w/ Gym + Doorman, near MSG by Blueground',
 'Private Room With Kitchen Across From MSG',
 'Accessible Two Bedded Suite With Kitchen near MSG',
 'Two Beds in Private Room near Times Square and MSG',
 'Private and Spacious Queen Room Across From MSG',
 'Comfy Kips Bay 1BR w/ Gym, Doorman + Sundecks, walk to MSG by Blueground',
 'A+ Chelsea Location! MSG, Javits, Penn!',
 'Heart of Williamsburg living for 1-5 msg!',
 '3 Bedroom by MSG + Empire State',
 'Next to Times Square/Javits/MSG! Amazing 1BR!']

In [9]:
#Q5: Find the room names containing "cinema" or "film"

unique(find_room_names("cinema") + find_room_names("film"))


['Cinema + gym included with room',
 'WoodyAllen FilmSet-Like Digs (Apt)',
 'Cool apartment in Brooklyn with free cinema & gym',
 'HUGE LUX 2FLOOR 2 BDRMSOHO LOFTw/HOME CINEMA',
 'WoodyAllen FilmSet-Like Digs (Room)',
 'Victorian Film location',
 'Shoot. Film. Sleep. Unique Loft Space in Brooklyn.',
 'Sunny private room featured in film',
 'The Otheroom Bar/Event/Filming Space -read details',
 'Music Recording Mixing Film Photography Art',
 'TV-PHOTO-FILM-CINEMA-ART GALLERY-MUSIC STUDIO-LOFT',
 'Premium Chelsea 1BR w/ Gym, W/D, Doorman, Sundeck, Cinema, by Blueground',
 'Cinema Studio on Duplex Apt.',
 'Modern Townhouse for Photo, Film &  Daytime Events',
 "Downtown Filmmaker's Loft by WTC",
 'Clean music/film themed bedroom',
 'Stunning Chelsea 1BR w/ Gym, W/D, Doorman, Sundeck, Cinema, by Blueground',
 'Brooklyn townhouse for filming',
 'Film Location',
 'Film / photography location in unique apartment']

In [10]:
#Q6: Which host names are anagrams of the word "aisle"?
def is_anagram_of_asile_check(host_name):
    return set(list(host_name.lower())) == {'a','i','s','l','e'} and len(host_name) == 5

unique(
    list(
        filter(
            is_anagram_of_asile_check, 
            remove_none(coerce_str(extract_column('host_name')))
        )
    )
)

['Leisa', 'Elias', 'Elisa']

In [11]:
#Q7: List all room ids that receive more than 15 reviews per month 

coerce_str(
    list(
        map(
            lambda x: x[0], 
            filter(lambda x: x[1] != None and x[1] > 15, 
                   zip(extract_column('room_id'), 
                       extract_column('reviews_per_month'))))))


['16276632',
 '18173787',
 '21550302',
 '22176831',
 '22750161',
 '28826608',
 '30423106',
 '31090222',
 '31249784',
 '32678719',
 '32678720']

In [12]:
#Q8: What percentage of rooms are entire homes or apartments? (room_type == "Entire home/apt")
len(
    list(
        filter(
            lambda x: x == "Entire home/apt",
            extract_column('room_type')
        )
    )
) / len(extract_column('room_id')) * 100


51.96645873811229

In [13]:
#Q9: Which room ids in Staten Island (`neighborhood_group == "Staten Island"`) received their last review in the year 2017 or earlier?

import datetime

def parse_date(s):
    return datetime.datetime.strptime(s, "%Y-%m-%d")


si = filter(lambda x: x[1] == "Staten Island",
       zip(extract_column('room_id'),
           extract_column('neighborhood_group'), 
           extract_column('last_review')))

si = filter(lambda x: x[2] != None, si)
si = map(lambda x: (x[0], x[1], parse_date(x[2]).year), si)
si = filter(lambda x: x[2] <= 2017, si)
si = map(lambda x: x[0], si)
coerce_str(si)




['712136',
 '738588',
 '826307',
 '1598033',
 '2794725',
 '8253376',
 '9045427',
 '12914239',
 '15949915',
 '15988392',
 '17475711']

In [14]:
def availability_per_host_name(host_name, neighborhood_group=None):
    ls = zip(extract_column('availability_365'),
           extract_column('neighborhood_group'), 
           extract_column('host_name'))
    ls = filter(lambda x: x[0] != None and x[2] != None, ls)
    ls = filter(lambda x: (neighborhood_group == None or x[1] == neighborhood_group), ls)
    ls = map(lambda x: (x[0], x[1], str(x[2])), ls)
    ls = filter(lambda x: x[2].lower() == host_name.lower(), ls)
    ls = map(lambda x: x[0], ls)
    return list(ls)

In [15]:
#Q10: What are the different availabilities of all rooms in the neighborhood group "Brooklyn" whose host name is "Stanley"?

sorted(unique(availability_per_host_name("Stanley", "Brooklyn")), reverse=True)




[360, 353, 348, 333, 321, 308, 264, 106, 15, 0]

In [16]:
#Q11: What is the difference between the most and least availability among all rooms whose host name is "Helena" (`host_name == "Helena"`)?

helena_availability = availability_per_host_name('Helena')
a = max(helena_availability) - min(helena_availability)
a





253

In [17]:
def find_prices_within(lat_min, lat_max, long_min, long_max):
    pricelist = zip(extract_column('price'),
           extract_column('latitude'), 
           extract_column('longitude'))
    pricelist = filter(lambda x: x[1]>=lat_min and x[1]<=lat_max, pricelist)
    pricelist = filter(lambda x: x[2]>= long_min and x[2]<= long_max, pricelist)
    return(list(map(lambda x: x[0], pricelist)))
    # TODO: create a list
    # TODO: add every price of rooms that locate in the given area to the list
    # TODO: return the filled list of prices

In [18]:
#Q12: What is the average price among rooms within (40.50 <= latitude <= 40.75, -74.00 <= longitude <= -73.95)?
mean(find_prices_within(40.50, 40.75, -74.00, -73.95))


169.4356387798248

In [19]:
#Q13: What is the highest price among rooms within (40.75 <= latitude <= 41.00, -73.95 <= longitude <= -73.85)?
max(find_prices_within(40.75, 41.00, -73.95, -73.85))

10000

In [20]:
#Q14: What percentage of rooms within (40.50 <= latitude <= 41.00, -74.00 <= longitude <= -73.85) 
# have a price less than 100?
len(list(filter(lambda x: x<100, find_prices_within(40.50, 41.00, -74.00, -73.85))))/len(find_prices_within(40.50, 41.00, -74.00, -73.85))*100



45.27971221694415

In [21]:
#Q15: What is the average ratio of the number of reviews to availability in the neighborhood Manhattan Beach?

# The denominator is the availability of a room (`availability_365` column). 
#    The numerator is the number of reviews of a room (`number_of_reviews` column).
# 2. Be careful! You need to compute the ratio for each room in the given neighborhood, 
#    then take the average of those ratios. 
#    Simply dividing the sum of reviews by the sum of availability
#    will calculate the wrong answer.
def avg_ratio_reviews_to_availability(neighborhood):
    list_avail = zip(extract_column('neighborhood'), extract_column('number_of_reviews'), extract_column('availability_365'))
    list_avail = filter(lambda x: x[2] != 0, list_avail)
    list_avail = filter(lambda x: x[0] != None and x[1] != None and x[2] != None, list_avail)
    list_avail = filter(lambda x: x[0] == neighborhood, list_avail)
    ratio_list = map(lambda x: x[1] / x[2], list_avail)
    return mean(ratio_list)

    
    
avg_ratio_reviews_to_availability('Manhattan Beach')    
    


0.27323293295076073

In [22]:
#Q16: What is the average ratio of the number of reviews to availability in the neighborhood Riverdale?
avg_ratio_reviews_to_availability('Riverdale')

0.043732334137215216

In [23]:
#Q17: Which neighborhood in the 
# neighborhood group Queens has the highest average ratio of the number of reviews to availability?
from functools import reduce

queens_nb = zip(extract_column('neighborhood'), extract_column('neighborhood_group'))
queens_nb = filter(lambda x: x[1] == "Queens", queens_nb)
queens_nb = map(lambda x: x[0], queens_nb)
queens_nb = unique(queens_nb)
reviews_ratio = map(lambda x: (x, avg_ratio_reviews_to_availability(x)), queens_nb)
reduce(lambda x, y: y if y[1] > x[1] else x, reviews_ratio)[0]


'Fresh Meadows'

In [24]:
#Q18: What percentage of rooms whose name contains the word "cozy" also contain the word "home" in its name?

len(secondary_word_in_found_rooms("cozy", "home"))/len(find_room_names("cozy")) * 100



3.9522598317354727

In [25]:
#Q19: What percentage of rooms whose name contains the word "pool" also contain the word "gym" in its name?

len(secondary_word_in_found_rooms("pool", "gym"))/len(find_room_names("pool")) * 100


48.92086330935252

In [26]:
#Q20: What is the minimum amount of money one needs to spend to stay for 10 days in Manhattan, and then 5 days in Staten Island?

z = list(zip(extract_column('neighborhood_group'), extract_column('price'), extract_column('availability_365'), extract_column('minimum_nights')))

manhattan = list(filter(lambda x: x[0] == "Manhattan" and x[2] >= 10 and x[3] <= 10, z))
si = list(filter(lambda x: x[0] == "Staten Island" and x[2] >= 5 and x[3] <= 5, z))

manhattan.sort(key=lambda x: x[1])
si.sort(key=lambda x: x[1])

manhattan[0][1] * 10 + si[0][1] * 5



175